In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
dig_mnist = pd.read_csv("./Dig-MNIST.csv")
sample_submission = pd.read_csv("./sample_submission.csv")

In [3]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head()

,id,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dig_mnist.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train.shape, test.shape, dig_mnist.shape

((60000, 785), (5000, 785), (10240, 785))

In [7]:
two_image = train[0:2000]

In [8]:
two_image

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def convert_dataset(data):
    images, labels = [], []
    for idx, row in data.iterrows():
        labels.append([int(row[0])])
        images.append(row[1:].values)
    return images, labels

In [16]:
train_images, train_labels = convert_dataset(train)
valid_images, valid_labels = convert_dataset(dig_mnist)
test_images, test_labels = convert_dataset(test)

In [17]:
import torch
import torch.nn as nn

In [20]:
tensor_train_images = torch.stack([torch.Tensor(i.reshape(28,28)) for i in train_images])
tensor_train_labels = torch.stack([torch.Tensor(i) for i in train_labels])

tensor_valid_images = torch.stack([torch.Tensor(i.reshape(28,28)) for i in valid_images])
tensor_valid_labels = torch.stack([torch.Tensor(i) for i in valid_labels])

tensor_test_images = torch.stack([torch.Tensor(i.reshape(28,28)) for i in test_images])
tensor_test_labels = torch.stack([torch.Tensor(i) for i in test_labels])

In [21]:
torch.save(tensor_train_images, "train_images.pt")
torch.save(tensor_train_labels, "train_labels.pt")

torch.save(tensor_valid_images, "valid_images.pt")
torch.save(tensor_valid_labels, "valid_labels.pt")

torch.save(tensor_test_images, "test_images.pt")
torch.save(tensor_test_labels, "test_labels.pt")

In [217]:
train_dataset = torch.utils.data.TensorDataset(tensor_train_images, tensor_train_labels)
valid_dataset = torch.utils.data.TensorDataset(tensor_valid_images, tensor_valid_labels)
test_dataset = torch.utils.data.TensorDataset(tensor_test_images, tensor_test_labels)

In [218]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128,shuffle=True)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=128)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

In [212]:
model = nn.Sequential(nn.Linear(784,256),nn.ReLU(), nn.Linear(256,10))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [229]:
EPOCHS = 20
for epoch in range(EPOCHS):
    print("Running Epoch: {}".format(epoch+1))
    model.train()
    total_loss = 0
    image_count = 0
    for X, y in trainloader:
        optimizer.zero_grad()
        y_pred = model(X)
        y = y.reshape(-1).long()
        loss = criterion(y_pred, y)
        total_loss+=loss.item()
        image_count+=X.shape[0]
        loss.backward()
        optimizer.step()
    print("Average Train Loss: {}".format(1.0*total_loss/image_count))
    
    model.eval()
    total_loss = 0
    image_count = 0
    for X, y in validloader:
        y_pred = model(X)
        y = y.reshape(-1).long()
        loss = criterion(y_pred, y)
        total_loss+=loss.item()
        image_count+=X.shape[0]
    print("Average Valid Loss: {}".format(1.0*total_loss/image_count))

Running Epoch: 1
Average Train Loss: 0.0002472410455967437
Average Valid Loss: 0.1561160683631897
Running Epoch: 2
Average Train Loss: 0.000307988496526883
Average Valid Loss: 0.14362462067510934
Running Epoch: 3
Average Train Loss: 0.0002709573554151469
Average Valid Loss: 0.14022381498944014
Running Epoch: 4
Average Train Loss: 0.0001927914014195106
Average Valid Loss: 0.14485887275077403
Running Epoch: 5
Average Train Loss: 0.00019662537434281762
Average Valid Loss: 0.17703359303995966
Running Epoch: 6
Average Train Loss: 0.00020166153405065708
Average Valid Loss: 0.17710627848282456
Running Epoch: 7
Average Train Loss: 0.00024686105162742566
Average Valid Loss: 0.20302088428288698
Running Epoch: 8
Average Train Loss: 0.00029145015012252645
Average Valid Loss: 0.18779156440868974
Running Epoch: 9
Average Train Loss: 0.00018722212192822513
Average Valid Loss: 0.17943509495817125
Running Epoch: 10
Average Train Loss: 0.00014065096772233575
Average Valid Loss: 0.19272778544109315
Runni

In [214]:
y_pred = model(tensor_images[2])

In [215]:
y_pred

tensor([-118.5649, -118.1250,  253.3491, -101.6456, -246.4227, -196.4479,
           2.1325,  -42.4619, -165.3644, -107.0228], grad_fn=<AddBackward0>)

In [216]:
sample_submission.head()

,id,label
0,0,5
1,1,5
2,2,5
3,3,5
4,4,5


In [223]:
preds = []
for X, y in testloader:
    y_pred = np.argmax(model(X).detach().numpy())
    preds.append(y_pred)

In [225]:
len(preds)

5000

In [226]:
sample_submission["label"] = pd.Series(preds)

In [228]:
sample_submission.to_csv("baseline_submission.csv", index=False)

NameError: name 'pd' is not defined